# Saving a movie

A code to save the evolution of your bathimetry data as an mp4.

In [9]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm
from matplotlib import animation

# Dredging pit marker positions
pit_x = np.array([7450.0, 7950.0, 8450.0, 8950.0]) + 25
pit_y = np.array([5452.0898, 5451.0, 5451.0, 5451.0]) - 30

def save_bathymetry_movie(nc_file, base_folder="outputs", subfolder="scenario", filename="bathymetry.mp4",
                           vmin=-15, vmax=5, vcenter=0, fps=5):
    # Create output directory
    output_path = os.path.join(base_folder, subfolder, filename)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Load NetCDF data
    dataset = xr.open_dataset(nc_file)
    xc = dataset["XCOR"].values
    yc = dataset["YCOR"].values
    depth = dataset["DPS"].values * -1  # Convert to elevation
    time_values = dataset["time"].values

    # Trim edges
    xc_inner = xc[1:-1, 1:-1]
    yc_inner = yc[1:-1, 1:-1]
    depth_inner = depth[:, 1:-1, 1:-1]

    # Custom colormap
    colors = [
        (0.00, "#000066"),
        (0.10, "#0000ff"),
        (0.30, "#00ffff"),
        (0.40, "#00ffff"),
        (0.50, "#ffffcc"),
        (0.60, "#ffcc00"),
        (0.75, "#cc6600"),
        (0.90, "#228B22"),
        (1.00, "#006400"),
    ]
    terrain_like = LinearSegmentedColormap.from_list("custom_terrain", colors)
    norm = TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)

    # Set up plot
    fig, ax = plt.subplots(figsize=(8, 6))
    mesh = ax.pcolormesh(xc_inner, yc_inner, depth_inner[0], shading='auto', cmap=terrain_like, norm=norm)
    fig.colorbar(mesh, ax=ax, label='Elevation relative to MSL [m]')
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    title = ax.set_title("")

    ax.scatter(pit_x, pit_y, marker='s', s=20, facecolors='none',
               edgecolors='black', linewidth=0.5, label='Dredging Locations')

    # Animation update function
    def update(frame):
        mesh.set_array(depth_inner[frame].ravel())
        title.set_text(f"Depth at {time_values[frame]} Morphological years")
        return mesh, title

    # Create animation
    ani = animation.FuncAnimation(fig, update, frames=len(time_values), blit=False)

    # Save animation
    ani.save(output_path, fps=fps, dpi=150)
    print(f"Animation saved as {output_path}")

    dataset.close()

Then use it like this:

In [14]:
trim5 = 'Mahakam_500/Dreding duration/Mhaa/trim-001.nc'      # Your trimfile here

save_bathymetry_movie(
        trim5,
        subfolder='Subfoldername',                # instert subfolder name
        filename=f"{trimIDs[i]}_simulation.mp4",   # Insert title of the movie
        fps=5                                     # modify vmin and vmax to your map 
    )

Thanks for reading\
Cheers,